In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
import torch.nn as nn
import torch
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F

In [2]:
np.random.seed(0)
data = pd.read_csv("..\\Data\\cle\\Rand_train.csv", sep=',')
data = shuffle(data).reset_index(drop=True)
data_val = pd.read_csv("..\\Data\\cle\\Rand_valid.csv", sep=',')

In [3]:
X = data.iloc[:,1:]
Y = data.iloc[:,0]
X_val = data_val.iloc[:,1:]
Y_val = data_val.iloc[:,0]

X2 = X.values
X2_val = X_val.values

In [4]:
cut0 = 0
cut1 = 1146
y10 = (Y == cut0) * 1
y11 = ((Y > cut0) & (Y <= cut1)) * 1
y12 = (Y > cut1) * 1
y21 = y11 * Y
y22 = y12 * Y
Y2 = np.stack((y10, y11, y12, y21, y22), axis=-1)


y10 = (Y_val == cut0) * 1
y11 = ((Y_val > cut0) & (Y_val <= cut1)) * 1
y12 = (Y_val > cut1) * 1
y21 = y11 * Y_val
y22 = y12 * Y_val
Y2_val = np.stack((y10, y11, y12, y21, y22), axis=-1)

In [5]:
from fit_nn import fit, w_comb_error , w_mae_error, class_error, loss_error, comb_error, mae_error

In [6]:
class Net1(nn.Module):
    def __init__(self):
        super(Net1, self).__init__()
        
        self.fc111 = nn.Linear(16, 24, bias = True)
        self.fc112 = nn.Linear(24, 1, bias = True)
        
        self.fc121 = nn.Linear(16, 24, bias = True)
        self.fc122 = nn.Linear(24, 1, bias = True)
        
        self.fc131 = nn.Linear(16, 24, bias = True)
        self.fc132 = nn.Linear(24, 1, bias = True)
       
        
        self.fc211 = nn.Linear(16, 24, bias=True)
        self.fc212 = nn.Linear(24, 5, bias=True)
        self.fc213 = nn.Linear(5, 1, bias=True)
        
        self.fc221 = nn.Linear(16, 24, bias=True)
        self.fc222 = nn.Linear(24, 5, bias=True)
        self.fc223 = nn.Linear(5, 1, bias=True)
        
        self.fc31 = nn.Linear(5,5, bias=False)       
        
               
    def softmax(self,x):
        expou = torch.exp(x)
        sm = expou/torch.sum(expou, dim=1).view(-1,1)
        return sm
        
    def disc_activation(self, h):
        v1 = self.softmax(h[:,:3])
        v2 = torch.exp(h[:,3:5])    
        return torch.cat([v1, v2], dim=1)
        
                
    def forward(self, x):
        x11 = torch.tanh(self.fc111(x))
        x11 = self.fc112(x11)
        
        x12 = torch.tanh(self.fc121(x))
        x12 = self.fc122(x12)
        
        x13 = torch.tanh(self.fc131(x))
        x13 = self.fc132(x13)                 
        
        #x21 = torch.tanh(self.fc211(x))
        x21 = self.fc211(x)
        x21 = F.relu(self.fc212(x21))
        x21 = self.fc213(x21)
        
        #x22 = torch.tanh(self.fc221(x))
        x22 = self.fc221(x)
        x22 = F.relu(self.fc222(x22))
        x22 = self.fc223(x22)
        
        x3 = torch.cat((x11, x12, x13, x21, x22), 1)        
        x3 = self.fc31(x3)
        x3 = self.disc_activation(x3)        
       
        return x3

In [7]:
n_epochs = 2500
lr = 0.0001
verbose = 1
n_batches = 6
batch_to_avg = 2
ep_to_check = 5
clipping = 0.20
#sig1 = 2 * 52000
#sig2 = 2 * 9500000
sig1 = 10000000
sig2 = 10000000
PATH = '..\\Data\\checkpoints\\mytraining10.pt'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [8]:
w0 = 1
w1 = 1
w2 = 1
def m_error(output, target):
    return w_comb_error(output, target, sig1, sig2, w0, w1, w2)

In [9]:
def m_error1(output, target):
    return comb_error(output, target, sig1, sig2)

In [10]:
w0 = 1.000
w1 = 0.531
w2 = 8.439
def m_error2(output, target):
    return w_mae_error(output, target, w0, w1, w2)

In [11]:
def m_loss(output, target):
    return loss_error(output, target, sig1, sig2)

In [12]:
def weights_init_normal(m):     
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.normal_(0, y)
        if m.bias is not None:
            m.bias.data.normal_(0, y)

In [13]:
def weights_init_uniform_rule(m):
    classname = m.__class__.__name__
    torch.manual_seed(0)
    if classname.find('Linear') != -1:
        n = m.in_features
        y = 1.0/np.sqrt(n)
        m.weight.data.uniform_(-y, y)
        m.bias.data.data.uniform_(-y, y)

In [14]:
net = Net1()
#optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.95)  
optimizer = optim.Adam(net.parameters(), lr=lr)

net.apply(weights_init_normal)
#net.apply(weights_init_uniform_rule)

#PATH = '..\\Data\\checkpoints\\mytraining10.pt'
#net.load_state_dict(torch.load(PATH, map_location = device))
    
a, b, c, d, e = fit(X=X2, X_val=X2_val, Y=Y2, Y_val=Y2_val, net=net, optimizer=optimizer, error=m_error1, val_error = mae_error,
                class_error = class_error, loss_error = m_loss, n_epochs=n_epochs, n_batches=n_batches, batch_to_avg=batch_to_avg, 
                    ep_to_check=ep_to_check, clipping=clipping, PATH=PATH, device=device, 
                                            verbose=verbose, min_val_loss = float('inf'))

#(PENDIENTE)

Epoch 1: Training Loss: 0.9634036719799042, Validation Loss: 289.6586608886719
Epoch 2: Training Loss: 0.9404637118180593, Validation Loss: 289.6611022949219
Epoch 3: Training Loss: 0.9200257857640585, Validation Loss: 289.6632385253906
Epoch 4: Training Loss: 0.9020120700200399, Validation Loss: 289.6650085449219
Epoch 5: Training Loss: 0.886344442764918, Validation Loss: 289.66607666015625
Epoch 6: Training Loss: 0.8728678921858469, Validation Loss: 289.66632080078125
New Checkpoint Saved into PATH
Epoch 7: Training Loss: 0.8614083230495453, Validation Loss: 289.66571044921875
Epoch 8: Training Loss: 0.8517058392365774, Validation Loss: 289.66412353515625
Epoch 9: Training Loss: 0.8434635102748871, Validation Loss: 289.6617431640625
Epoch 10: Training Loss: 0.8363347252209982, Validation Loss: 289.65869140625
Epoch 11: Training Loss: 0.8299453556537628, Validation Loss: 289.6551208496094
New Checkpoint Saved into PATH
Epoch 12: Training Loss: 0.8239718377590179, Validation Loss: 289.

Epoch 96: Training Loss: 0.656609445810318, Validation Loss: 277.4748840332031
New Checkpoint Saved into PATH
Epoch 97: Training Loss: 0.6562257905801138, Validation Loss: 277.3169250488281
Epoch 98: Training Loss: 0.6558368504047394, Validation Loss: 277.1642761230469
Epoch 99: Training Loss: 0.6554429829120636, Validation Loss: 277.0174255371094
Epoch 100: Training Loss: 0.6550538738568624, Validation Loss: 276.876220703125
Epoch 101: Training Loss: 0.6546764274438223, Validation Loss: 276.74383544921875
New Checkpoint Saved into PATH
Epoch 102: Training Loss: 0.6543169021606445, Validation Loss: 276.6122741699219
Epoch 103: Training Loss: 0.6539749801158905, Validation Loss: 276.4833679199219
Epoch 104: Training Loss: 0.6536887089411417, Validation Loss: 276.3607177734375
Epoch 105: Training Loss: 0.6534041861693064, Validation Loss: 276.2412414550781
Epoch 106: Training Loss: 0.6531239946683248, Validation Loss: 276.1271667480469
New Checkpoint Saved into PATH
Epoch 107: Training L

Epoch 191: Training Loss: 0.6379758020242056, Validation Loss: 270.0201721191406
New Checkpoint Saved into PATH
Epoch 192: Training Loss: 0.6378565828005472, Validation Loss: 269.9830017089844
Epoch 193: Training Loss: 0.637735386689504, Validation Loss: 269.9479064941406
Epoch 194: Training Loss: 0.637613981962204, Validation Loss: 269.91278076171875
Epoch 195: Training Loss: 0.6374943057696024, Validation Loss: 269.8773193359375
Epoch 196: Training Loss: 0.6373817225297292, Validation Loss: 269.83599853515625
New Checkpoint Saved into PATH
Epoch 197: Training Loss: 0.6372647881507874, Validation Loss: 269.7947082519531
Epoch 198: Training Loss: 0.6371538241704305, Validation Loss: 269.7572326660156
Epoch 199: Training Loss: 0.6370414594809214, Validation Loss: 269.7221374511719
Epoch 200: Training Loss: 0.6369290252526602, Validation Loss: 269.68853759765625
Epoch 201: Training Loss: 0.6368177831172943, Validation Loss: 269.64990234375
New Checkpoint Saved into PATH
Epoch 202: Traini

Epoch 285: Training Loss: 0.6294670303662618, Validation Loss: 267.7542724609375
Epoch 286: Training Loss: 0.6293965776761373, Validation Loss: 267.7381896972656
New Checkpoint Saved into PATH
Epoch 287: Training Loss: 0.6293260256449381, Validation Loss: 267.7210388183594
Epoch 288: Training Loss: 0.6292563776175181, Validation Loss: 267.70343017578125
Epoch 289: Training Loss: 0.6291875541210175, Validation Loss: 267.68560791015625
Epoch 290: Training Loss: 0.6291216115156809, Validation Loss: 267.6678161621094
Epoch 291: Training Loss: 0.62905419866244, Validation Loss: 267.6507263183594
New Checkpoint Saved into PATH
Epoch 292: Training Loss: 0.6289858420689901, Validation Loss: 267.6336975097656
Epoch 293: Training Loss: 0.6289167404174805, Validation Loss: 267.6163330078125
Epoch 294: Training Loss: 0.6288477977116903, Validation Loss: 267.5990905761719
Epoch 295: Training Loss: 0.628779262304306, Validation Loss: 267.5819396972656
Epoch 296: Training Loss: 0.6287111937999725, Va

Epoch 378: Training Loss: 0.6238914628823599, Validation Loss: 266.765625
Epoch 379: Training Loss: 0.6238398551940918, Validation Loss: 266.7569580078125
Epoch 380: Training Loss: 0.6237853666146597, Validation Loss: 266.74688720703125
Epoch 381: Training Loss: 0.6237291991710663, Validation Loss: 266.73577880859375
New Checkpoint Saved into PATH
Epoch 382: Training Loss: 0.6236723562081655, Validation Loss: 266.724365234375
Epoch 383: Training Loss: 0.6236171722412109, Validation Loss: 266.71533203125
Epoch 384: Training Loss: 0.6235637962818146, Validation Loss: 266.7098693847656
Epoch 385: Training Loss: 0.6235154867172241, Validation Loss: 266.7041015625
Epoch 386: Training Loss: 0.6234647234280905, Validation Loss: 266.6971740722656
New Checkpoint Saved into PATH
Epoch 387: Training Loss: 0.6234118441740671, Validation Loss: 266.6890869140625
Epoch 388: Training Loss: 0.6233574549357096, Validation Loss: 266.679931640625
Epoch 389: Training Loss: 0.6233031551043192, Validation Lo

...

Epoch 1749: Training Loss: 0.5848309795061747, Validation Loss: 291.06201171875
Epoch 1750: Training Loss: 0.5848238070805868, Validation Loss: 291.077880859375
Epoch 1751: Training Loss: 0.584837794303894, Validation Loss: 291.03076171875
Epoch 1752: Training Loss: 0.5848324000835419, Validation Loss: 291.031005859375
Epoch 1753: Training Loss: 0.5847994287808737, Validation Loss: 291.0723571777344
Epoch 1754: Training Loss: 0.5847747226556143, Validation Loss: 291.0207214355469
Epoch 1755: Training Loss: 0.584761917591095, Validation Loss: 290.9952087402344
Epoch 1756: Training Loss: 0.5847630699475607, Validation Loss: 291.0175476074219
Epoch 1757: Training Loss: 0.5847544471422831, Validation Loss: 291.0485534667969
Epoch 1758: Training Loss: 0.5847427050272623, Validation Loss: 291.0284118652344
Epoch 1759: Training Loss: 0.5847267607847849, Validation Loss: 290.9916687011719
Epoch 1760: Training Loss: 0.5847103993097941, Validation Loss: 291.0615234375
Epoch 1761: Training Loss: 

Epoch 1850: Training Loss: 0.583758552869161, Validation Loss: 291.3377685546875
Epoch 1851: Training Loss: 0.5837493638197581, Validation Loss: 291.26531982421875
Epoch 1852: Training Loss: 0.5837200184663137, Validation Loss: 291.2666320800781
Epoch 1853: Training Loss: 0.5837031801541647, Validation Loss: 291.2843933105469
Epoch 1854: Training Loss: 0.5836873650550842, Validation Loss: 291.26251220703125
Epoch 1855: Training Loss: 0.5836788415908813, Validation Loss: 291.2388610839844
Epoch 1856: Training Loss: 0.5836763183275858, Validation Loss: 291.2702941894531
Epoch 1857: Training Loss: 0.5836794972419739, Validation Loss: 291.2738037109375
Epoch 1858: Training Loss: 0.5836645364761353, Validation Loss: 291.2532958984375
Epoch 1859: Training Loss: 0.5836373468240103, Validation Loss: 291.29595947265625
Epoch 1860: Training Loss: 0.5836312174797058, Validation Loss: 291.34967041015625
Epoch 1861: Training Loss: 0.5836401979128519, Validation Loss: 291.34716796875
Epoch 1862: Tra

Epoch 1951: Training Loss: 0.5826912720998129, Validation Loss: 291.62457275390625
Epoch 1952: Training Loss: 0.5826884210109711, Validation Loss: 291.6202087402344
Epoch 1953: Training Loss: 0.5826604068279266, Validation Loss: 291.65264892578125
Epoch 1954: Training Loss: 0.5826429526011149, Validation Loss: 291.6239013671875
Epoch 1955: Training Loss: 0.5826392471790314, Validation Loss: 291.63336181640625
Epoch 1956: Training Loss: 0.5826388796170553, Validation Loss: 291.6810302734375
Epoch 1957: Training Loss: 0.5826320846875509, Validation Loss: 291.64410400390625
Epoch 1958: Training Loss: 0.5826082428296407, Validation Loss: 291.6417541503906
Epoch 1959: Training Loss: 0.5825885832309723, Validation Loss: 291.7088928222656
Epoch 1960: Training Loss: 0.582588662703832, Validation Loss: 291.7215270996094
Epoch 1961: Training Loss: 0.582606315612793, Validation Loss: 291.6869812011719
Epoch 1962: Training Loss: 0.5825961530208588, Validation Loss: 291.6647644042969
Epoch 1963: Tr

Epoch 2050: Training Loss: 0.5816873808701833, Validation Loss: 291.8064880371094
Epoch 2051: Training Loss: 0.5816630224386851, Validation Loss: 291.8022155761719
Epoch 2052: Training Loss: 0.5816232562065125, Validation Loss: 291.79180908203125
Epoch 2053: Training Loss: 0.5816042224566141, Validation Loss: 291.8169860839844
Epoch 2054: Training Loss: 0.581618736187617, Validation Loss: 291.8433837890625
Epoch 2055: Training Loss: 0.5816337466239929, Validation Loss: 291.82012939453125
Epoch 2056: Training Loss: 0.5816051860650381, Validation Loss: 291.7869873046875
Epoch 2057: Training Loss: 0.5815660754839579, Validation Loss: 291.8121337890625
Epoch 2058: Training Loss: 0.5815577805042267, Validation Loss: 291.8334045410156
Epoch 2059: Training Loss: 0.5815641184647878, Validation Loss: 291.8081970214844
Epoch 2060: Training Loss: 0.5815601944923401, Validation Loss: 291.810546875
Epoch 2061: Training Loss: 0.5815507968266805, Validation Loss: 291.8544616699219
Epoch 2062: Trainin

KeyboardInterrupt: 

### Evaluate on Validation

In [ ]:
PATH = '..\\Data\\checkpoints\\mytraining10.pt'
device = torch.device('cpu')
net = Net1()
net = net.to(device)
net.load_state_dict(torch.load(PATH, map_location = device))

In [ ]:
with torch.no_grad():
    net.eval()
    
    val_inputs = torch.FloatTensor(X_val)
    val_labels = torch.FloatTensor(Y_val)
    val_inputs, val_labels = val_inputs.to(device), val_labels.to(device) 
    
    val_outputs = net.forward(val_inputs)
            
    val_outputs2, val_labels2 = val_outputs2.numpy(), val_labels2.numpy()

Evaluate sepparate results from the model

In [ ]:
plt.plot(b,'b')
plt.axvline(x=1006, c='r')
plt.show()

In [ ]:
plt.plot(d,'b')
plt.axvline(x=1006, c='r')
plt.show()

In [ ]:
plt.plot(e,'b')
plt.axvline(x=1006, c='r')
plt.show()

In [ ]:
19000000
10000000

In [ ]:
2 * 9500000

In [ ]:
Epoch 1006: Training Loss: 0.6030789613723755, Validation Loss: 263.94036865234375, Test loss: 270.42722